In [69]:
import pandas as pd

from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [70]:
path_to_data_folder =  '/Users/alexraudvee/Desktop/TU_e/year_2/Q3Y2_Process_Mining/process_mining_data'

In [78]:
# extract the 1-10 lags

df_2017 = pd.read_csv(f"{path_to_data_folder}/BPI_2017_naive.csv")

df_2017['concept:name - lag_1'] = df_2017['concept:name'].shift(1)
df_2017['concept:name - lag_2'] = df_2017['concept:name'].shift(2)
df_2017['concept:name - lag_3'] = df_2017['concept:name'].shift(3)

df_2017 = df_2017.loc[3:]

# define target

df_2017['concept:name next'] = df_2017['concept:name'].shift(-1)

/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_24584/3807258033.py:3: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2017 = pd.read_csv(f"{path_to_data_folder}/BPI_2017_naive.csv")


In [79]:
# test slices
df_2017 = df_2017.loc[:1000]

In [80]:
df_2017

,Unnamed: 0,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,...,time_to_next_event,time_to_next_event_seconds,predicted_time_to_next_event_seconds,elapsed_time_from_start,predicted_start_time,predicted_step,concept:name - lag_1,concept:name - lag_2,concept:name - lag_3,concept:name next
3,3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392,Existing loan takeover,New credit,...,0 days 00:00:00.011000,0.01,35.17,0 days 00:01:21.088000,5 days 16:10:11.014999397,W_Handle leads,W_Handle leads,A_Submitted,A_Create Application,W_Complete application
4,4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403,Existing loan takeover,New credit,...,0 days 00:00:00.010000,0.01,0.01,0 days 00:01:21.099000,5 days 08:22:12.629814592,W_Complete application,W_Handle leads,W_Handle leads,A_Submitted,A_Concept
5,5,statechange,User_1,A_Concept,Application,ApplState_642383566,complete,2016-01-01 09:52:36.413,Existing loan takeover,New credit,...,4 days 23:34:08.079000,430448.08,179447.74,0 days 00:01:21.109000,5 days 05:51:35.263530324,A_Concept,W_Complete application,W_Handle leads,W_Handle leads,A_Create Application
6,6,Created,User_1,A_Create Application,Application,Application_1691306052,complete,2016-01-01 10:16:11.500,Home improvement,New credit,...,0 days 00:00:00.049000,0.05,0.04,0 days 00:00:00,4 days 18:38:55.010316100,A_Create Application,A_Concept,W_Complete application,W_Handle leads,A_Submitted
7,7,statechange,User_1,A_Submitted,Application,ApplState_284636842,complete,2016-01-01 10:16:11.549,Home improvement,New credit,...,0 days 00:00:00.191000,0.19,0.30,0 days 00:00:00.049000,4 days 18:50:11.306445745,A_Submitted,A_Create Application,A_Concept,W_Complete application,W_Handle leads
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,996,statechange,User_1,A_Submitted,Application,ApplState_826687676,complete,2016-01-05 13:28:07.003,Home improvement,New credit,...,0 days 00:00:00.373000,0.37,0.30,0 days 00:00:00.034000,4 days 18:50:11.306445745,A_Submitted,A_Create Application,A_Concept,W_Complete application,W_Handle leads
997,997,Created,User_1,W_Handle leads,Workflow,Workitem_1696440271,schedule,2016-01-05 13:28:07.376,Home improvement,New credit,...,0 days 00:02:29.719000,149.72,35.17,0 days 00:00:00.407000,5 days 03:53:50.679851693,W_Handle leads,A_Submitted,A_Create Application,A_Concept,W_Handle leads
998,998,Deleted,User_1,W_Handle leads,Workflow,Workitem_405441961,withdraw,2016-01-05 13:30:37.095,Home improvement,New credit,...,0 days 00:00:00.027000,0.03,35.17,0 days 00:02:30.126000,5 days 16:10:11.014999397,W_Handle leads,W_Handle leads,A_Submitted,A_Create Application,W_Complete application
999,999,Created,User_1,W_Complete application,Workflow,Workitem_473295245,schedule,2016-01-05 13:30:37.122,Home improvement,New credit,...,0 days 00:00:00.007000,0.01,0.01,0 days 00:02:30.153000,5 days 08:22:12.629814592,W_Complete application,W_Handle leads,W_Handle leads,A_Submitted,A_Concept


In [81]:
df_2017.columns

Index(['Unnamed: 0', 'Action', 'org:resource', 'concept:name', 'EventOrigin',
       'EventID', 'lifecycle:transition', 'time:timestamp', 'case:LoanGoal',
       'case:ApplicationType', 'case:concept:name', 'case:RequestedAmount',
       'FirstWithdrawalAmount', 'NumberOfTerms', 'Accepted', 'MonthlyCost',
       'Selected', 'CreditScore', 'OfferedAmount', 'OfferID',
       'timestamp_difference', 'event_seq', 'time_to_next_event',
       'time_to_next_event_seconds', 'predicted_time_to_next_event_seconds',
       'elapsed_time_from_start', 'predicted_start_time', 'predicted_step',
       'concept:name - lag_1', 'concept:name - lag_2', 'concept:name - lag_3',
       'concept:name next'],
      dtype='object')

In [82]:
# Prepare data
X = df_2017[['concept:name - lag_2', 'concept:name - lag_1', 'concept:name']]
y = df_2017['concept:name next']

# Use LabelEncoder to convert string labels to numerical values
label_encoder = LabelEncoder()
X['concept:name - lag_1'] = label_encoder.fit_transform(X['concept:name - lag_1'])
X['concept:name - lag_2'] = label_encoder.fit_transform(X['concept:name - lag_2'])
X['concept:name'] = label_encoder.fit_transform(X['concept:name'])
y = label_encoder.fit_transform(y)

# Define the Random Forest Classifier
rf_clf = RandomForestClassifier(n_jobs=-1)

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [19, 20, 21],
    'max_depth': [18, 19, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform nested cross-validation
outer_cv = KFold(n_splits=5, shuffle=False, random_state=None)  # 5-fold outer cross-validation
inner_cv = KFold(n_splits=3, shuffle=False, random_state=None)  # 3-fold inner cross-validation

grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, scoring='accuracy', cv=inner_cv)
grid_search.fit(X, y)

# Get the best model
best_model = grid_search.best_estimator_

# Print the results
print(f"Best params: {grid_search.best_params_}")
print(f'Nested Cross-Validation Accuracy: {grid_search.best_score_:.4f}')


/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_24584/2066021085.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['concept:name - lag_1'] = label_encoder.fit_transform(X['concept:name - lag_1'])
/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_24584/2066021085.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['concept:name - lag_2'] = label_encoder.fit_transform(X['concept:name - lag_2'])
/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_24584/2066021085.py:9: SettingW

Best params: {'max_depth': 18, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 19}
Nested Cross-Validation Accuracy: 0.9529


In [83]:
# Predict labels using the best model
df_2017['predicted_next_event'] = best_model.predict(X)

# decoding
df_2017['predicted_next_event'] = label_encoder.inverse_transform(df_2017['predicted_next_event'])

In [84]:
df_2017

,Unnamed: 0,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,...,time_to_next_event_seconds,predicted_time_to_next_event_seconds,elapsed_time_from_start,predicted_start_time,predicted_step,concept:name - lag_1,concept:name - lag_2,concept:name - lag_3,concept:name next,predicted_next_event
3,3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392,Existing loan takeover,New credit,...,0.01,35.17,0 days 00:01:21.088000,5 days 16:10:11.014999397,W_Handle leads,W_Handle leads,A_Submitted,A_Create Application,W_Complete application,W_Complete application
4,4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403,Existing loan takeover,New credit,...,0.01,0.01,0 days 00:01:21.099000,5 days 08:22:12.629814592,W_Complete application,W_Handle leads,W_Handle leads,A_Submitted,A_Concept,A_Concept
5,5,statechange,User_1,A_Concept,Application,ApplState_642383566,complete,2016-01-01 09:52:36.413,Existing loan takeover,New credit,...,430448.08,179447.74,0 days 00:01:21.109000,5 days 05:51:35.263530324,A_Concept,W_Complete application,W_Handle leads,W_Handle leads,A_Create Application,A_Create Application
6,6,Created,User_1,A_Create Application,Application,Application_1691306052,complete,2016-01-01 10:16:11.500,Home improvement,New credit,...,0.05,0.04,0 days 00:00:00,4 days 18:38:55.010316100,A_Create Application,A_Concept,W_Complete application,W_Handle leads,A_Submitted,A_Submitted
7,7,statechange,User_1,A_Submitted,Application,ApplState_284636842,complete,2016-01-01 10:16:11.549,Home improvement,New credit,...,0.19,0.30,0 days 00:00:00.049000,4 days 18:50:11.306445745,A_Submitted,A_Create Application,A_Concept,W_Complete application,W_Handle leads,W_Handle leads
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,996,statechange,User_1,A_Submitted,Application,ApplState_826687676,complete,2016-01-05 13:28:07.003,Home improvement,New credit,...,0.37,0.30,0 days 00:00:00.034000,4 days 18:50:11.306445745,A_Submitted,A_Create Application,A_Concept,W_Complete application,W_Handle leads,W_Handle leads
997,997,Created,User_1,W_Handle leads,Workflow,Workitem_1696440271,schedule,2016-01-05 13:28:07.376,Home improvement,New credit,...,149.72,35.17,0 days 00:00:00.407000,5 days 03:53:50.679851693,W_Handle leads,A_Submitted,A_Create Application,A_Concept,W_Handle leads,W_Handle leads
998,998,Deleted,User_1,W_Handle leads,Workflow,Workitem_405441961,withdraw,2016-01-05 13:30:37.095,Home improvement,New credit,...,0.03,35.17,0 days 00:02:30.126000,5 days 16:10:11.014999397,W_Handle leads,W_Handle leads,A_Submitted,A_Create Application,W_Complete application,W_Complete application
999,999,Created,User_1,W_Complete application,Workflow,Workitem_473295245,schedule,2016-01-05 13:30:37.122,Home improvement,New credit,...,0.01,0.01,0 days 00:02:30.153000,5 days 08:22:12.629814592,W_Complete application,W_Handle leads,W_Handle leads,A_Submitted,A_Concept,A_Concept
